### Display the weather stations on a map

In [ ]:
# Load packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import folium
# import geopandas as gpd (not working)

In [7]:
import warnings
warnings.filterwarnings("ignore")

In [8]:
# Load data from the csv file

df_forecast = pd.read_csv('data/forecast_weather.csv', keep_default_na=False)

pd.set_option('display.max_columns', 81)
df_forecast.tail(5)

,latitude,longitude,origin_datetime,hours_ahead,temperature,dewpoint,cloudcover_high,cloudcover_low,cloudcover_mid,cloudcover_total,10_metre_u_wind_component,10_metre_v_wind_component,data_block_id,forecast_datetime,direct_solar_radiation,surface_solar_radiation_downwards,snowfall,total_precipitation
3424507,59.7,26.2,2023-05-30 02:00:00,48,10.150049,5.662744,0.061066,0.257339,0.004059,0.308594,6.890626,-3.258144,637,2023-06-01 02:00:00,0.0,0.0,0.0,0.0
3424508,59.7,26.7,2023-05-30 02:00:00,48,10.259790,5.922754,0.107880,0.895203,0.198700,0.906036,6.890626,-3.768032,637,2023-06-01 02:00:00,0.0,0.0,0.0,0.0
3424509,59.7,27.2,2023-05-30 02:00:00,48,10.467798,6.072534,0.000000,0.387466,0.448853,0.530518,6.214112,-4.700405,637,2023-06-01 02:00:00,0.0,0.0,0.0,0.0
3424510,59.7,27.7,2023-05-30 02:00:00,48,10.694116,7.009058,0.000000,0.505936,0.997314,0.997498,5.810060,-4.338345,637,2023-06-01 02:00:00,0.0,0.0,0.0,0.0
3424511,59.7,28.2,2023-05-30 02:00:00,48,11.233179,7.069238,0.703461,0.292313,0.934021,0.989716,2.719728,-0.922939,637,2023-06-01 02:00:00,0.0,0.0,0.0,0.0


In [9]:
col_order = ['origin_datetime', 'forecast_datetime','hours_ahead', 'latitude', 'longitude', 'data_block_id','temperature', 'dewpoint', 'cloudcover_high', 'cloudcover_low',
       'cloudcover_mid', 'cloudcover_total', '10_metre_u_wind_component','10_metre_v_wind_component', 'direct_solar_radiation', 'surface_solar_radiation_downwards',
       'snowfall', 'total_precipitation']

df_forecast = df_forecast[col_order]
df_forecast.head()

,origin_datetime,forecast_datetime,hours_ahead,latitude,longitude,data_block_id,temperature,dewpoint,cloudcover_high,cloudcover_low,cloudcover_mid,cloudcover_total,10_metre_u_wind_component,10_metre_v_wind_component,direct_solar_radiation,surface_solar_radiation_downwards,snowfall,total_precipitation
0,2021-09-01 02:00:00,2021-09-01 03:00:00,1,57.6,21.7,1,15.655786,11.553613,0.904816,0.019714,0.000000,0.905899,-0.411328,-9.106137,0.0,0.0,0.0,0.0
1,2021-09-01 02:00:00,2021-09-01 03:00:00,1,57.6,22.2,1,13.003931,10.689844,0.886322,0.004456,0.000000,0.886658,0.206347,-5.355405,0.0,0.0,0.0,0.0
2,2021-09-01 02:00:00,2021-09-01 03:00:00,1,57.6,22.7,1,14.206567,11.671777,0.729034,0.005615,0.000000,0.730499,1.451587,-7.417905,0.0,0.0,0.0,0.0
3,2021-09-01 02:00:00,2021-09-01 03:00:00,1,57.6,23.2,1,14.844507,12.264917,0.336304,0.074341,0.000626,0.385468,1.090869,-9.163999,0.0,0.0,0.0,0.0
4,2021-09-01 02:00:00,2021-09-01 03:00:00,1,57.6,23.7,1,15.293848,12.458887,0.102875,0.088074,0.000015,0.176590,1.268481,-8.975766,0.0,0.0,0.0,0.0


In [10]:
count_per_location = df_forecast.groupby(['latitude', 'longitude']).size().reset_index(name='count')
pd.set_option('display.max_rows', 112)
# Print the resulting DataFrame
count_per_location.head()

,latitude,longitude,count
0,57.6,21.7,30576
1,57.6,22.2,30576
2,57.6,22.7,30576
3,57.6,23.2,30576
4,57.6,23.7,30576


In [11]:
count_per_location.shape

(112, 3)

### Map creation using Geopandas (deinstalled after success)

#### Code used to generate weather station map

In [ ]:
# Load GeoJSON file with county boundaries
estonia_counties_geojson = 'data/estonia-with-regions_.geojson'
counties_geojson = gpd.read_file(estonia_counties_geojson)

# Convert 'latitude' and 'longitude' columns into tuples
gps_locations = list(zip(count_per_location['latitude'], count_per_location['longitude']))

# Add a new column 'station' to count_per_location with station names
count_per_location['station'] = ['station{}'.format(i + 1) for i in range(len(count_per_location))]

# Create a map centered around the first GPS coordinate
map_center = gps_locations[0]
map_obj = folium.Map(location=map_center, zoom_start=7)

# Iterate through features and add GeoJSON layer for county boundaries with labels
for feature in counties_geojson.iterfeatures():
    county_name = feature['properties']['name']
    folium.GeoJson(
        data=feature,
        name='geojson',
        style_function=lambda x: {'color': 'black', 'fillOpacity': 0.0, 'weight': 1},
        highlight_function=lambda x: {'weight': 3},
        tooltip=f'{county_name}',
    ).add_to(map_obj)

# Add markers for each GPS location with popup information
for location, station, count in zip(gps_locations, count_per_location['station'], count_per_location['count']):
    popup_text = f'Station: {station}<br>Count: {count}'
    folium.Marker(location, popup=folium.Popup(popup_text, parse_html=True)).add_to(map_obj)

# Save the map to an HTML file or display it
map_obj.save('gps_locations_map_with_popup_and_counties.html')